**British Airways Revenue Forecasting 3** <br>
Oxford Saïd Business School <br>
Analysis 3: Regressions <br><br>

*Jake Schneider* <br>
MBA Candidate <br>
November 2020 <br>

----

**Load Libraries**

In [1]:
#Import libraries
import sys
import pandas as pd
from datetime import date, datetime, time, timedelta
import pendulum
import json
import requests
import numpy as np
from numpy import NaN
from numpy import nanmean
import math

import collections

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='ticks', context='talk')

from matplotlib.offsetbox import AnchoredText
from sklearn import linear_model
from sklearn.metrics import r2_score
import statsmodels.api as sm
#import fbprophet

import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import datapane as dp

import warnings
import itertools

----

**Load Clean BA Data Set**

In [2]:
# Load CFA Data from CFA_analysis_1

df = pd.read_csv('../3_Outputs/Data/ba_df.csv')

*Concat Macro Data with BA Income Statement*

---

**Regressions**